In [1]:
import pandas as pd
import sys
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
print(widgets.__version__)

7.5.1


In [2]:
%matplotlib inline

In [3]:
widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

FileUpload(value={}, description='Upload')

In [4]:
tab = widgets.Tab()    
out = widgets.Output(layout={'border': '1px solid black'})
up = widgets.FileUpload(accept="", multiple=False)

In [5]:
delim = widgets.RadioButtons(
    options=[';', ',', ' '],
    description='Separator: ',
    disabled=False)

eraser = widgets.SelectMultiple(
    options=['tab','"'],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False)

eraser

SelectMultiple(description='Eraser: ', index=(0,), options=('tab', '"'), value=('tab',))

In [6]:
rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
rows

IntSlider(value=0, continuous_update=False, description='# of lines:')

In [7]:
button_upload = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='warning',
    tooltip='Click to Upload',
    icon='check')
button_upload 

Button(button_style='warning', description='Upload', icon='check', style=ButtonStyle(), tooltip='Click to Uplo…

In [8]:
button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')
button_preview 

Button(button_style='info', description='Preview', icon='search', style=ButtonStyle(), tooltip='Click to Previ…

In [9]:
button_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='danger',
    tooltip='Click to Plot',
    icon='pencil')
button_plot

Button(button_style='danger', description='Plot', icon='pencil', style=ButtonStyle(), tooltip='Click to Plot')

In [10]:
graph_type = widgets.Dropdown(
    options=['Bar Chart', 'Line Chart'],
    value='Bar Chart',
    description='Chart Type:',
    disabled=False)
graph_type

Dropdown(description='Chart Type:', options=('Bar Chart', 'Line Chart'), value='Bar Chart')

In [11]:
x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False)
x_axis

Dropdown(description='X-Axis:', options=('',), value='')

In [12]:
y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Y-Axis:',
    disabled=False)
y_axis

Dropdown(description='Y-Axis:', options=('',), value='')

In [13]:
color_picker = widgets.ColorPicker(
    concise=False,
    description='Color Picker: ',
    value='lightblue',
    disabled=False)
color_picker

ColorPicker(value='lightblue', description='Color Picker: ')

In [14]:
toggle = widgets.ToggleButtons(
    options=['Preview  ', 'Info  ', 'Stats  '],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info', 'tachometer'])
toggle

ToggleButtons(button_style='warning', description='Options', icons=('search', 'info', 'tachometer'), options=(…

In [15]:
accordion = widgets.Accordion(children=[
    up,
    widgets.VBox([delim, eraser]), 
    rows])
accordion.set_title(0, 'File Selection')
accordion.set_title(1, 'Delimiter')
accordion.set_title(2, 'Skip Rows')
accordion

Accordion(children=(FileUpload(value={}, description='Upload'), VBox(children=(RadioButtons(description='Separ…

In [16]:
accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_preview, button_upload]),
    out
])
accordion_box

In [17]:
children = [
    accordion_box, 
    widgets.VBox([toggle, out]),
    widgets.VBox([
        widgets.HBox([graph_type, color_picker]),
        widgets.HBox([x_axis, y_axis]), 
        button_plot,
        out
])]
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Describer")
tab.set_title(2, "Plotter")

In [18]:
def content_parser():
    if up.value == {}:
        with out:
            print('No CSV loaded')    
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]["metadata"]["type"]
            if typ == "text/csv":
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')
    
            if eraser.value != {}: 
                for val in eraser.value:
                    if val == "tab":
                        content_str = content_str.replace("\t","")
                    else:
                        content_str = content_str.replace(val,"")
            if content_str != "":
                str_io = StringIO(content_str) 
                return str_io
def df_converter():
    content = content_parser()
    if content is not None:
        df = pd.read_csv(content, sep=delim.value, index_col=False, skiprows=rows.value)    
        return df
    else:
        return None
def preview():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n -----Now this is how your DF looks like:----- \n')
        if df is not None:
            print(df.head(10))
        else:
            print('Configuration is wrong/missing...')
def upload():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n --------Your uploaded DF looks like:-------- \n')
        if df is not None:
            print(df)
            x_axis.options = df.columns
            y_axis.options = df.columns
        else:
            print('Configuration is wrong/missing...')
def desc():
    info_level = toggle.value
    if info_level != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(
                info_level))
            if df is not None:
                if info_level == 'Info  ':
                    print(df.info(verbose=True))
                elif info_level == 'Stats  ':
                    print(df.describe())
                elif info_level == 'Preview  ':
                    print(df.head(5))
                else:
                    print('Configuration is wrong/missing...')
    
    
def plot():
    graph = graph_type.value
    if graph != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(
                graph))
            if (df is not None):
                df = df.head(5)
                height = df[y_axis.value]
                bars = df[x_axis.value]
                y_pos = np.arange(len(height))
                plt.figure(figsize=(10,4))
                if graph == 'Bar Chart':
                    plt.bar(
                        y_pos, 
                        height, 
                        color=color_picker.value)
                    plt.xticks(y_pos, bars)
                elif graph == 'Line Chart':
                    plt.plot(
                        bars,
                        height, 
                        color=color_picker.value,
                        marker='o', 
                        linestyle='solid'
                    )
                    plt.xticks(bars)
                plt.show()
                

In [19]:
def preview_clicked(b):
    preview()
def upload_clicked(b):
    upload()
def desc_clicked(b):
    desc()
def plotter_clicked(b):
    plot()
    
button_preview.on_click(preview_clicked)
button_upload.on_click(upload_clicked)
toggle.observe(desc_clicked, 'value')
button_plot.on_click(plotter_clicked)

In [20]:
tab

In [21]:
# can select multiple
toggle_buttons= {'a':widgets.ToggleButton(description='a', value=False),
          'b':widgets.ToggleButton(description='b', value=False),
          'c':widgets.ToggleButton(description='c', value=False),
          'd':widgets.ToggleButton(description='d', value=False)}

toggle_disp = widgets.VBox([
                widgets.HBox([toggle_buttons['a'], toggle_buttons['b']], description='htest'),
                widgets.HBox([toggle_buttons['c'], toggle_buttons['d']])
                            ], description='test')

def on_toggle(**toggles):
    for name, obj in toggles.items():
        print(name, obj)
    # print(toggles)

interact_out = widgets.interactive_output(on_toggle, toggle_buttons)
display(toggle_disp, interact_out)

Output()

In [22]:
# intext = widgets.IntText(min=1, max=100, step=1, continuous_update=True)
# 
# widgets.jslink((tb_fruits, 'value'), (intext, 'value'))
# widgets.VBox([play, slider, out])

In [23]:
# only one
fruits = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
tb_fruits = widgets.ToggleButtons(options = fruits, style={'button_width':'100px'}, layout={'width': '320px'})
tb_fruits 

ToggleButtons(layout=Layout(width='320px'), options=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'), style=ToggleButt…

In [25]:
widgets.ToggleButtons(
    options=[('Slow', '11'), ('Regular', '22'), ('Fast', '33')],
    description='Speed:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

ToggleButtons(button_style='info', description='Speed:', options=(('Slow', '11'), ('Regular', '22'), ('Fast', …